In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from model import Model

Using TensorFlow backend.


# Pre Processing

In [2]:
df = pd.read_csv('dog_breed/labels.csv')
occurrencies_df = df.pivot_table(index='breed', aggfunc=len).sort_values('id', ascending=False)

In [3]:
# Get N breeds with most occurrency
N = 35
labels = np.array(occurrencies_df.index[:35])
df = df.loc[df['breed'].isin(labels)]

In [4]:
train_path = 'dog_breed/train/'
data = list()
labels = list()
image_shape = (128, 128)

for file in os.listdir(train_path):
    filename = file.split('.')[0]
    
    if df['id'].str.contains(filename).any():
        image = cv2.imread(train_path + file)
        image = cv2.resize(image, image_shape)
        data.append(image)
        
        label = df.loc[df['id'] == filename, 'breed'].iloc[0]
        labels.append(label)
        
data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [5]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=1, stratify=labels)

In [6]:
data_augmentation = ImageDataGenerator(rotation_range=25, width_shift_range=0.25, height_shift_range=0.25, \
                                       shear_range=0.2, zoom_range=0.25, horizontal_flip=True, \
                                       fill_mode='nearest')